# PHF TEST for SOM Energia indexed invoicing
set module path in PYTHONPATH , i.e. /home/erp/src/giscedata_facturacio_indexada_som

# Configuration
  * **tariff_name**: Tarifa i.e 2.0A, 3.0A ...
  * **start_date_str**: Initial invoice date at 01:00:00 (yyyy-mm-dd)
  * **end_date_str**: Final invoice date at 00:00:00 (yyyy-mm-dd)
  * **curve_origin**: 'tpl' or 'profile'

## versions 
Not ESIOS params
  
  * **imu**: Impost Municipal en [%]
  * **d**: Desviaments [€/kWh]
  * **k**: Coeficient / Marge comercial [€/kWh]
  * **omie**: Cuota omie/ree [€/MWh]
  * **fe**: Fons Eficiència [€/MWh]
  * **atr**: Diccionari Peatges per periode {P1: 0, P2: ... , Pn: ...} en [€/kWh]
  * **pc**: Diccionari Pagos Capacidad per periode {P1: 0, P2: ... , Pn: ...} en [€/kWh]

## Profiling
Data to profile by REE coeficients

  * **profile_consume**: Diccionari amb consum a perfilar per períodes {P1: , P2:, ..., Pn:} en [kWh] 

## File (TPL)
  * **filename**: ruta al fitxer amb la corba
  El format ha de ser:
    [data/hora];[valor]
   
  On 
  * **data/hora**: no s'utilitza però serveix per l'usuari
  * **valor**: consum horari en [W]
  
  Hi ha d'haver una línia per hora , 24 hores per dia (i.e 30 dies x 24 h = 720)
  

In [ ]:
tariff_name = '6.1A'
start_date_str = '2017-04-01'
end_date_str = '2017-05-01'
#curve_origin = 'profile'
curve_origin = 'tpl'
versions = dict(
    imu = 1.5,
    d = 0.00025,
    k = 0.002,
    omie = 0.13341,
    fe = 0.27,
    atr = {'P1': 0.018762, 'P2': 0.012575, 'P3': 0.004670, 'P4': 0.0018762, 'P5': 0.012575, 'P6': 0.004670},
    pc = {'P1': 0.008374, 'P2': 0.004304, 'P3': 0.000058, 'P4': 0.008374, 'P5': 0.004304, 'P6': 0.000058},
)
profile_consume = {'P1': 1000, 'P2': 2000, 'P3': 3000, 'P4': 300, 'P5': 600, 'P6': 900}
#curve_filename = '/home/index/corbes/corba.csv'
curve_filename = '/home/index/corbes/vorgran201804.csv'

In [ ]:
from tarifes import *
import copy
from datetime import datetime, date
from enerdata.datetime.holidays import get_holidays
from enerdata.contracts.tariff import *
from enerdata.profiles.profile import *
from enerdata.metering.measure import *
from profile import REEInitalProfile
import csv
import copy
from calendar import monthrange

In [ ]:
from libfacturacioatr.pool.component import *

In [ ]:
peatge = {
    '3.0A': {'P1': 0.018762, 'P2': 0.012575, 'P3': 0.004670, 'P4': 0.0018762, 'P5': 0.012575, 'P6': 0.004670},
    '6.1A': {'P1': 0.026674, 'P2': 0.019921, 'P3': 0.010615, 'P4': 0.005283, 'P5': 0.003411, 'P6': 0.002137}
}
pagos_capacidad = {
    '3.0A': {'P1': 0.008374, 'P2': 0.004304, 'P3': 0.000058, 'P4': 0.008374, 'P5': 0.004304, 'P6': 0.000058},
    '3.1A': {'P1': 0.006432, 'P2': 0.003463, 'P3': 0.000000, 'P4': 0.006432, 'P5': 0.003463, 'P6': 0.0},
    '6.1A': {'P1': 0.006432, 'P2': 0.002969, 'P3': 0.001979, 'P4': 0.001484, 'P5': 0.001484, 'P6': 0.0},
    '6.1B': {'P1': 0.006432, 'P2': 0.002969, 'P3': 0.001979, 'P4': 0.001484, 'P5': 0.001484, 'P6': 0.0},
}
tariff_classes = {
    '3.0A': Tarifa30APoolSOM,
    '3.1A': Tarifa31APoolSOM,
    '6.1A': Tarifa61APoolSOM,
    '6.1B': Tarifa61BPoolSOM,
}
atr = peatge[tariff_name]
pc = pagos_capacidad[tariff_name]
esios = '67c6aff80ca331eec78e1f62b7ffc6799e2674d82d57c04104a612db43496db3'
# calc Holidays
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
HOLIDAYS = []
for d in [start_date, end_date]:
    HOLIDAYS.extend(get_holidays(start_date.year))
HOLIDAYS = set(HOLIDAYS)
HOLIDAYS
TariffClass = tariff_classes[tariff_name]

In [ ]:
curve_factor = 1000.0 # W: 1 kW: 1000
print start_date
print end_date
versions

In [ ]:
## Profiling code

TARIFFS = {
    '2.0A': T20A,
    '2.0DHA': T20DHA,
    '2.0DHS': T20DHS,
    '2.1A': T21A,
    '2.1DHA': T21DHA,
    '2.1DHS': T21DHS,
    '3.0A': T30A,
    '3.1A': T31A
}
def array_padding(array, length, value=0):
    return array + ([value] * (length - len(array)))

def get_cofs(start, end):
    c = Coefficients()

    while start <= end:
        cofs = REEProfile.get(start.year, start.month)

        c.insert_coefs(cofs)
        # Add one month but always day 1 to avoid month skip, ie
        # 2015-11-04 > 2015-11-02
        start += relativedelta(months=1, day=1)

    return c

def get_profiled_curve(start, end, tariff, consumes):
    t = TARIFFS[tariff]()
    
    measures = []
    curve_start = datetime.combine(start_date, datetime.min.time())
    curve_end = datetime.combine(end_date - relativedelta(days=1), datetime.min.time())
    for period, consume in consumes.items():
        # start 
        measures.append(
             EnergyMeasure(
                 start_date.date(),
                 TariffPeriod(period, 'te'), 0 * 1000,
                 consumption=0 * 1000
             )
        )
        # end 
        measures.append(
             EnergyMeasure(
                 curve_end.date(),
                 TariffPeriod(period, 'te'), consume * 1000,
                 consumption=consume * 1000
             )
        )

        coefs = get_cofs(curve_start, curve_end)
        profiler = Profiler(coefs)
        
        consums = {}
        drag = 'period'
        
        for hour, vals in profiler.profile(t, measures, drag_method=drag):
            d = hour.date()
            # Hacky patch to work for both systems REE Style number of hours
            # and Esios style datetimes
            first_hour = getattr(profiler.coefficient, 'first_hour', 1)
            if first_hour == 1:
                if hour.hour == 0:
                    d -= timedelta(days=1)
            consums.setdefault(d, [])
            consums[d].append(vals['aprox'])
        # print consums
        curve = []
        for day,hours in sorted(consums.items()):
            curve.append(array_padding(hours, 25))
            
    return curve

In [ ]:
## TPL code 
def get_tpl_curve(filename):
    curve_data = []
    day = []
    with open(filename, 'rb') as tplfile:
        curve = csv.reader(tplfile, delimiter=';')
        hour = 0
        for line in curve:
            day.append(int(line[1]))
            if hour < 23:
                hour+=1
            else:
                hour=0
                day.append(0)
                curve_data.append(day)
                day = copy.copy([])

        return curve_data
                
#curve_data = []
# fixe
#curve_data = [[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 0, 1000, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0], [0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 0, 1000, 0, 1000, 0, 0], [1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 1000, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 3000, 3000, 2000, 3000, 0, 0, 0], [0, 1000, 0, 0, 1000, 0, 1000, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 2000, 3000, 3000, 2000, 1000, 0, 0], [0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 0, 1000, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 3000, 3000, 2000, 3000, 0, 0, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 3000, 0, 0, 0], [0, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 0, 0, 0, 1000, 0, 0], [1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 1000, 0, 0, 0, 0, 0, 0], [0, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 2000, 3000, 3000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0], [1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0], [1000, 0, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 3000, 3000, 3000, 2000, 1000, 0, 0], [0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 0, 1000, 0, 1000, 0, 0], [0, 1000, 0, 0, 1000, 0, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 1000, 0, 0], [0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 3000, 2000, 3000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0], [1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0], [0, 1000, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 2000, 3000, 2000, 0, 0, 0], [1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 0], [0, 0, 1000, 0, 0, 0, 1000, 0, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 0, 0], [0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 2000, 0, 0, 0], [1000, 0, 1000, 0, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 3000, 2000, 3000, 2000, 0, 0, 0]]
#[[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 1000, 0, 0],[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 2000, 3000,3000,2000, 0, 0, 0],[0, 1000, 0, 1000, 0, 0, 1000, 1000, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0],[0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000,0,0, 0, 0, 1000, 0, 1000, 0, 0],[1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 1000, 0, 0],[0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0,0,0, 3000, 3000, 3000, 2000, 1000, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 3000, 2000, 3000, 2000, 0, 1000, 0],[1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000,0,0, 1000, 0, 0, 1000, 0, 3000, 3000, 3000, 2000, 0, 0, 0],[1000, 0, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 2000, 2000, 0, 0, 0],[0, 1000, 0, 1000, 0, 1000,0,1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 3000, 3000, 2000, 3000, 0, 0, 0],[0, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 1000, 0, 0],[0, 1000, 0, 0, 1000,0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 1000, 0],[0, 1000, 0, 1000, 0, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 2000, 3000, 3000, 2000, 0, 0, 0],[0, 1000, 0, 0,1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 2000, 3000, 0, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 2000, 3000, 3000, 2000, 1000, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 3000, 3000, 2000, 3000, 0, 0, 0],[1000, 0, 1000, 0, 0, 1000, 0, 1000, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 1000, 2000, 3000,3000, 2000, 0, 0, 0],[0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0, 0, 1000, 0],[0, 1000, 0, 0, 0, 1000, 0, 0, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 0, 0,1000, 0, 0, 0],[1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 3000, 3000, 2000, 2000, 1000, 0, 0],[0, 1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0,3000, 3000, 3000, 2000, 0, 1000, 0],[0, 1000, 0, 1000, 0, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 3000, 2000, 3000, 2000, 0, 0, 0],[0, 1000, 0, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0,1000, 0, 0, 1000, 3000, 3000, 3000, 2000, 0, 0, 0],[1000, 0, 1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 2000, 3000, 3000, 2000, 0, 1000, 0],[1000, 0, 0, 0, 1000, 0, 0, 1000, 0, 0,1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 0, 0, 0, 0, 0],[0, 0, 1000, 0, 0, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 0, 1000, 0, 0, 0, 1000, 0],[1000, 0, 1000, 0, 1000, 0, 0, 1000, 0, 0,1000, 0, 0, 1000, 0, 0, 1000, 0, 3000, 3000, 2000, 2000, 0, 0, 0],[1000, 0, 0, 1000, 0, 1000, 0, 1000, 1000, 0, 0, 1000, 0, 0, 1000, 0, 0, 1000, 3000, 3000, 3000, 2000, 0, 0, 0]]
# generada
#for day in range(0, 28):
#    day = copy.copy([1000.0] * 24)
#    day.append(0.0)
#    curve_data.append(day)
     

In [ ]:
curve_end = end_date - relativedelta(days=1)
if curve_origin == 'tpl':
    curve_data = get_tpl_curve(curve_filename)
if curve_origin == 'profile':
    curve_data = get_profiled_curve(start_date, end_date, tariff_name, profile_consume)    

print curve_data
curve = Curve(start_date, curve_origin.upper())
curve.load(curve_data)
consums = {
    'activa': {start_date_str: {'start': start_date,'end': curve_end,'profile': curve_data}},
    'reactiva': {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5, 'P6': 6 }
}
print curve.total_sum

In [ ]:
tarifa = TariffClass(
    consums, {},
    start_date_str, end_date_str,
    facturacio=1, facturacio_potencia='max',
    data_inici_periode=start_date_str,
    data_final_periode=end_date_str,
    potencies_contractades={'P1': 20, 'P2': 20, 'P3': 20},
    versions={start_date_str: versions},
    holidays=HOLIDAYS,
    esios_token=esios,
    audit=['curve', 'pmd', 'phf', 'pc3_ree', 'perdues'],
)
tarifa.update_dates_consums(
    tarifa.data_inici,
    tarifa.data_final
)

In [ ]:
phf = tarifa.phf_calc(curve, start_date)
str(phf)

In [ ]:
print "curve total: {} PHF total: {}".format(curve.total_sum, phf.total_sum)

In [ ]:
last_month_day = monthrange(start_date.year, start_date.month)[1]
file_prefix = "{0}_{1}".format(
    datetime.strftime(start_date,'%Y%m%d'),
    datetime.strftime(datetime(start_date.year, start_date.month, last_month_day) ,'%Y%m%d'))
prgpncur = Prgpncur('C2_prgpncur_{0}'.format(file_prefix), esios)
prmncur = Prmncur('C2_prmncur_{0}'.format(file_prefix), esios)
perdclass_name = tarifa.perdclass.name
perdues = tarifa.perdclass('C2_{0}_{1}'.format(perdclass_name,file_prefix), esios)

In [ ]:
dia = 1                              # 1 a fi mes 
hora = 1                             # 0 a 25 (p.e. les 10:00 -> 9)
period_name = 'P3'                   # Període de la la hora seleccionada
IMU = versions['imu'] * 0.01         # en %
PMD = prmncur.get(dia, hora) * 0.001      # en €/MWh
PC3_REE = prgpncur.get(dia, hora) * 0.001     # en €/MWh
PC3 = versions['pc'][period_name]           # en €/kWh
OMIE_REE = versions['omie'] * 0.001  # en €/MWh
PERD = perdues.get(dia, hora) * 0.01      # en %
K = versions['k']            # en €/kWh
D = versions['d']            # en €/kWh
FONDO_EFI = versions['fe'] * 0.001   # en €/MWh
PA = versions['atr'][period_name]    # en €/kWh

PHF = (1 + IMU) * ((PMD - PC3_REE + PC3 + OMIE_REE) * (1 + PERD) + K + D + FONDO_EFI ) + PA
'{PHF} = (1 + {IMU}) * (({PMD} - {PC3_REE} + {PC3} + {OMIE_REE}) * (1 + {PERD}) + {K} + {D} + {FONDO_EFI}) + {PA}'.format(**locals())

In [ ]:
print '{0} €/kWh = {1} € / {2} kWh'.format(phf.get(dia, hora) / (curve.get(dia, hora) / curve_factor), phf.get(dia, hora) ,(curve.get(dia, hora) / curve_factor))  

In [ ]:
res = tarifa.factura_energia()
res

In [ ]:
from libfacturacioatr.pool.tarifes import aggr_version_comp
if tarifa.aggr_active:
    linies = aggr_version_comp(res[start_date_str])
else:
    linies = res[start_date_str]

In [ ]:
for k in sorted(linies.keys()):
    print("{0}: {1} kWh".format(k,sum(linies[k]['consum'])))
                                

In [ ]:
for k in sorted(linies.keys()):
    print("{0}: {1} kWh".format(k,sum(linies[k]['cost'])))

In [ ]:
for k in sorted(linies.keys()):
    print("{0}: {1} kWh".format(k,sum(linies[k]['cost']) / (sum(linies[k]['consum']) or 1.0 )))

In [ ]:
tarifa.get_audit_data('pmd')

In [ ]:
tarifa.get_audit_data('curve')

In [ ]:
tarifa.get_audit_data('pc3_ree')

In [ ]:
tarifa.get_audit_data('perdues')

In [ ]:
tarifa.get_audit_data('phf')

In [ ]:
tarifa.dump_audit_data('pmd', '/home/index/corbes/PMD.txt')

In [ ]:
tarifa.dump_audit_data('pc3_ree', '/home/index/corbes/PC3_REE.txt')

In [ ]:
tarifa.dump_audit_data('perdues', '/home/index/corbes/PERDUES.txt')

In [ ]:
tarifa.dump_audit_data('curve', '/home/index/corbes/CURVE.txt')

In [ ]:
tarifa.dump_audit_data('phf', '/home/index/corbes/PHF.txt')

In [ ]:
tarifa.consums['activa']

In [ ]:
len(curve_data[0])